In [4]:
import torch
import torch.nn as nn
import math
import numpy as np

np.random.seed(42)

In [5]:
class RequestType:
    def __init__(self, request_type, service_rate, arrival_rate, source, sink, distribution, switch_rate=None):
        # distribution is 1x2 if elastic and 1x1 if static
        
        self.type = request_type
        self.service_rate = service_rate
        self.arrival_rate = arrival_rate
        self.source = source
        self.sink = sink
        self.distribution = distribution
        self.switch_rate = switch_rate

class Request:
    def __init__(self, request_type, service_time, arrival_time, source, sink, transfer_rate, distribution=None):
        self.type = request_type
        self.service_time = service_time
        self.arrival_time = arrival_time
        self.source = source
        self.sink = sink
        self.bw = transfer_rate
        self.request_type = request_type
        
        if request_type == "elastic":
            self.distribution = distribution
            
    def get_encoding(self, nodes_in_environment):
        # as per our notes, this SHOULD return 1x5 tensor,
        # but we have one hot encodings INSIDE this tensor,
        # so we will flatten this and return, so the size will be
        # larger than 1x5
        
        # nodes_in_environment is a list of all the nodes in our graph
        # eg ["a", "b", "c"]
        
        # request is [one hot source, one hot destination, bw, service time, one hot type]
                
        one_hot_source = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.source)]), num_classes=len(nodes_in_environment))
        one_hot_dest   = nn.functional.one_hot(torch.tensor([nodes_in_environment.index(self.sink)]), num_classes=len(nodes_in_environment))

        if self.request_type == "static":
            one_hot_type = torch.tensor([[1, 0]])
        elif self.request_type == "elastic":
            one_hot_type = torch.tensor([[0, 1]])
            
        encoding = torch.cat((one_hot_source, 
                             one_hot_dest,
                             torch.tensor([[self.bw]]), # develop fix for when bw is already a tensor (when we have elastic request)
                             torch.tensor([[self.service_time]]),
                             one_hot_type), axis=1)
        
        return encoding

In [6]:
class Link:
    def __init__(self, node_1, node_2, bw_capacity):
        self.serving_requests = []
        self.nodes = [node_1, node_2]
        self.total_bw = bw_capacity
        
    def reset(self):
        self.serving_requests = []
        
    def add_request(self, request_obj):
        self.serving_requests.append(request_obj)
        
    def remaining_bw(self): 
        # subtracting bw being used from total bw capacity
        bw_being_used = 0
        for req in self.serving_requests:
            bw_being_used += req.bw
            
        return (self.total_bw - bw_being_used)

In [7]:
class Environment:
    # requests_in_service_encoder = nn.RNN(????, 7)
    
    def __init__(self, nodes, links, request_blueprints):
        """
        nodes: list of strings where each string is just a name or identifier of a node
        links: list of tuples where in tuple t, t[0] is first node, t[1] is another node, and t[2] is bw capacity of the link
        request_blueprints: list of DeploymentRequest objects
        """
        self.nodes = nodes
        self.links = {}
        self.request_history = []
        self.request_blueprints = request_blueprints
        
        for link in links:
            if link[0] not in self.nodes or link[1] not in self.nodes:
                raise Exception("Node in link " + str(link) + " doesn't exist")
            
            link_obj = Link(*link)

            self.links[link[0] + link[1]] = link_obj
            self.links[link[1] + link[0]] = link_obj
            
            
    def add_request(self, request): # we want to add this request to a link
        self.links[request.source + request.sink].add_request(request)
        self.request_history.append(request)
        print(self.links[request.source + request.sink])
    
    def reset(self):
        self.nodes = nodes
        for link in self.links.values():
            link.reset()
        
        # also return initial observation, implement this later once we have function
        # for encoding network
        
    def reward(self, request, decision):
        base_rate = 1         # 1 when static
        type_bonus = 0.9      # 0.9 when static
        if request.type == "elastic":
            base_rate = request.base_rate  
            type_bonus = 1.1                # 1.1 when elastic
            
        r = request.bandwidth * base_rate * request.service_time * type_bonus
        
        # if remaining bandwidth on link < 0, very "bad" reward
        remaining_bw = self.links[request.nodes[0] + request.nodes[1]].remaining_bw()
        if remaining_bw < 0:
            return (-r * 10)
        
        if decision == "accept":
            return r
        
        if decision == "reject":
            if request.type == "static":
                return 0
            elif request.type == "elastic":
                past_distributions = []
                for req in self.request_history:
                    if req.request_type == "elastic":
                        past_distributions.append(req.distribution)
                
                average_past_distribution = torch.mean(past_distributions, dim=1)
                current_req_distribution = torch.tensor(request.distribution)
                
                if bool(average_past_distribution[0] < current_req_distribution[0]):
                    return -1 * r * math.exp(-nn.functional.kl_div(average_past_distribution, current_req_distribution))
                else:
                    return 0
                
    #def step(self, action):
        # what happens if we have two requests that come in on the same timestep but there is only enough bandwidth for one?
        # do we the decision on the second request with knowledge of the first request
        # essentially, after we accept the first request, will we submit an updated encoding of the network to the policy network?
 
        # actions is a Nx2 matrix where the first column in the request and second is the decision
        # decision is either "accept" or "reject"
        
        

    def get_encoding(self):
        links_processed = [] 
        # these will store links that we have already encoded so we don't encode them again
        
        current_encoding = []
        
        # h = torch.zeros(7) # assuming 7 for h0 size
        # last_out = None
        
        env_encoding = []
        
        for link in self.links.values():
            if link in links_processed:
                continue

                        
            # Commented because we don't want to encode any queue for phase 1
            
            # for req in link.serving_requests
                # request is [one hot source, one hot destination, bw, service time, one hot type]
                
                # one_hot_source = nn.functional.one_hot(torch.tensor([self.nodes.index(req.source)]), num_classes=len(self.nodes))
                # one_hot_dest   = nn.functional.one_hot(torch.tensor([self.nodes.index(req.sink)]), num_classes=len(self.nodes))

                # req_tensor = torch.Tensor([]) # mismatched dimensions??!
                # last_out, h = self.requests_in_service_encoder(req_tensor, h)

            # current_encoding.append(torch.cat(torch.Tensor([link.remaining_bw]), last_out))
            # torch.stack(current_encoding)
            
            # check implementation later
            
            env_encoding.append(link.remaining_bw())
            
            links_processed.append(link)
            
        return torch.tensor(env_encoding)
    
    def create_requests():
        requests = []
        
        for request_type in self.request_blueprints:
            arrival_times = []
            service_times = []
            last_arrival = 0
            episode_timesteps = 600
        
            while last_arrival < episode_timesteps: # we want to generate requests till we reach episode end
                last_arrival += np.random.exponential(request_type.arrival_rate)
                arrival_times.append(last_arrival)
                
            for _ in arrival_times:
                service_times.append(np.random.exponential(request_type.service_rate))
                
            for arrival_time, service_time in zip(arrival_times, service_times):
                # start creating requests
                
                new_request = Request(request_type.type, service_time, arrival_time, request_type.source, request_type.sink, request_type.distribution[0])
                
                if request_type.type == "elastic": 
                    # we will start with the first distribution element as starting bw
                    # WE ASSUME that distribution[0] < distribution[1]
                    timesteps_from_deployment = 0
                    current_bw = request_type.distribution[0]
                    while timesteps_from_deployment < service_time:
                        if current_bw == request_type.distribution[0]:    
                            # we want to generate a scale request to increase bw
                            scale_bw = request_type.distribution[1] - current_bw
                            scale_service_time = np.random.exponential(request_type.switch_rate[0])
                            scale_request = Request(request_type.type, scale_service_time, \
                                                    last_arrival + timesteps_from_deployment, request_type.source, \
                                                   request_type.sink, scale_bw)
                            requests.append(scale_request)
                            
                            timesteps_from_deployment += scale_service_time
                            current_bw = request_type.distribution[0] + scale_bw # also equal to request_type.distribution[1]
                        elif current_bw == request_type.distribution[1]:
                            # we want to go to lower bw and spend some time there
                            time_spent_on_lower_bw = np.random.exponential(request_type.switch_rate[1])
                            timesteps_from_deployment += time_spent_on_lower_bw
                            current_bw = request_type.distribution[0]
                            
        # sort requests by arrival time
        requests.sort(key=lambda x: x.arrival_time)
        return requests

In [3]:
env = Environment(["a", "b", "c", "d", "e", "f"], [["a", "b", 10], ["a", "c", 10], ["b", "d", 10], \
                                                   ["c", "d", 0]
                                                   
                                                   [RequestType("static", 5, 5, "a", "b", [], [])])


TypeError: __init__() missing 1 required positional argument: 'request_blueprints'

In [148]:
env.links # notice that there are pairs that point to the same link obj

{'ab': <__main__.Link at 0x1123602e0>,
 'ba': <__main__.Link at 0x1123602e0>,
 'ac': <__main__.Link at 0x112360df0>,
 'ca': <__main__.Link at 0x112360df0>,
 'bc': <__main__.Link at 0x112360370>,
 'cb': <__main__.Link at 0x112360370>}

In [149]:
env.get_encoding() # this should return the remaining bandwidth on all links

tensor([17, 20, 15])

In [150]:
env.links['ab'].serving_requests[0].get_encoding(env.nodes) # sample encoding of a request

torch.Size([1, 3])
torch.Size([1, 1])
tensor([[1, 0, 0]])
tensor([5])


tensor([[1, 0, 0, 0, 1, 0, 5, 5, 1, 0]])

In [31]:
d = torch.tensor([[-1, 1], [1, 1]]).float()
bool(torch.mean(d, dim=1)[0] < torch.tensor([0.01, 1])[0])


True

In [ ]:
observation, reward, done, info = env.step(action)

In [19]:
np.random.seed(42)
print(np.random.exponential(5))
print(np.random.exponential(5))

2.3463404498842957
15.050607154587606
